In [14]:
import math
import geopy.distance
from dis import dis
import math
import geopandas as gpd
import numpy
from shapely import wkt
from shapely import wkb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
%matplotlib inline


train_data = pd.read_csv('data/stores_train.csv')
busstops = pd.read_csv('data/busstops_norway.csv')
grunnkrets_age = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_households = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('data/grunnkrets_income_households.csv')
grunnkrets_stripped = pd.read_csv('data/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')

In [15]:
train_data.head()

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528


In [16]:
#convert y values to logarithmic scale
train_data['revenue'] = np.log1p(train_data['revenue'])
train_data.head()
    

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,2.944334
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,3.211972
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,2.839020
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,2.331755
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,1.709826


In [17]:
#convert them back to normal scale
train_data['revenue'] = np.exp(train_data['revenue']) - 1
train_data.head()

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,18.998
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,24.828
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,17.099
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,10.296
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,5.528


In [ ]:
# Replace NaN in mall_name and chain_name columns with 'No mall' and 'No chain'
train_data.mall_name = train_data.mall_name.fillna('No mall')
train_data.chain_name = train_data.chain_name.fillna('No chain')

# Dummy variable for mall or no mall
train_data.loc[train_data['mall_name'].str.contains("No mall", na=False),'mall_dummy'] = 0
train_data.loc[~(train_data['mall_name'].str.contains("No mall", na=False)),'mall_dummy'] = 1
train_data.drop(['mall_name'],axis=1, inplace=True)

# 'store_name', 'year', 'sales_channel_name', 'address' columns are redundant, remove them
train_data = train_data.drop('store_name',axis=1)
train_data = train_data.drop('year',axis=1)
train_data = train_data.drop('sales_channel_name',axis=1)
train_data = train_data.drop('address',axis=1)


In [ ]:
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'municipality_name']], on='grunnkrets_id', how='left')
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

In [ ]:
chains = train_data.filter(['chain_name','revenue'], axis=1)
chains = chains.groupby('chain_name').mean()
chains = chains.rename(columns={'revenue':'mean_revenue_for_chain'})
train_data.merge(chains, how="left", on=["chain_name"])


In [ ]:
train_data = pd.merge(train_data, plaace_hierarchy[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')


In [ ]:
grunnkrets_age = grunnkrets_age.drop_duplicates(subset=['grunnkrets_id'], keep='last') # if there is value for 2016 we keep it, otherwise 2015
grunnkrets_age = grunnkrets_age.fillna(0)
grunnkrets_age = grunnkrets_age.drop('year',axis=1)
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(str)
grunnkrets_age['total_nbr_people'] = grunnkrets_age.sum(axis=1) # total number of inhabitants
grunnkrets_age['group1'] = grunnkrets_age.iloc[:,1:11].sum(axis=1) # 0-9 years old
grunnkrets_age['group2'] = grunnkrets_age.iloc[:,11:21].sum(axis=1) # 10-19 years old etc
grunnkrets_age['group3'] = grunnkrets_age.iloc[:,21:31].sum(axis=1)
grunnkrets_age['group4'] = grunnkrets_age.iloc[:,31:41].sum(axis=1)
grunnkrets_age['group5'] = grunnkrets_age.iloc[:,41:51].sum(axis=1)
grunnkrets_age['group6'] = grunnkrets_age.iloc[:,51:61].sum(axis=1)
grunnkrets_age['group7'] = grunnkrets_age.iloc[:,61:71].sum(axis=1)
grunnkrets_age['group8'] = grunnkrets_age.iloc[:,71:81].sum(axis=1)
grunnkrets_age['group9'] = grunnkrets_age.iloc[:,81:92].sum(axis=1) # 80-90 years old
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(int)
train_data = pd.merge(train_data, grunnkrets_age[['grunnkrets_id', 'total_nbr_people', 'group1', 'group2', 'group3', 'group4', 'group5', 'group6', 'group7', 'group8', 'group9']], on='grunnkrets_id', how='left')


In [ ]:
# Number of people per store in each grunnkrets regardless of hierarchy
number_stores = train_data['grunnkrets_id'].value_counts().rename_axis('grunnkrets_id').reset_index(name='store_counts_total') # Not including NaN (stores without a grunnkrets_id)
grunnkrets_stripped = pd.merge(grunnkrets_stripped, number_stores[['grunnkrets_id', 'store_counts_total']], on='grunnkrets_id', how='left')
grunnkrets_stripped.store_counts_total = grunnkrets_stripped.store_counts_total.fillna(0)
grunnkrets_stripped = pd.merge(grunnkrets_stripped, grunnkrets_age[['grunnkrets_id', 'total_nbr_people']], on='grunnkrets_id', how='left')
grunnkrets_stripped['nbr_people_per_store_in_grunnkrets'] = grunnkrets_stripped['total_nbr_people']/grunnkrets_stripped['store_counts_total']
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'nbr_people_per_store_in_grunnkrets']], on='grunnkrets_id', how='left')


In [ ]:
# Number of people per store in each grunnkrets in lv2
counts = train_data[["store_id", "grunnkrets_id", "lv2"]].groupby(
    ["grunnkrets_id", "lv2"]
).count().reset_index()
counts.columns = ["grunnkrets_id", "lv2", "counts_gr_lv2"]
train_data.merge(counts, how="left", on=["grunnkrets_id", "lv2"])

In [ ]:
train_data.update(train_data[['total_nbr_people','group1','group2','group3','group4','group5','group6','group7','group8','group9']].fillna(0))
#train_data.update(train_data[['nbr_people_per_store_in_grunnkrets','nbr_people_per_km2']].fillna(0))


In [ ]:
# Mean revenue for each lv2 in each municipality
municipalities = train_data[["municipality_name", "lv2", "revenue"]].groupby(
    ["municipality_name", "lv2"]
).mean().reset_index()
municipalities = municipalities.rename(columns={'revenue':'mean_revenue_for_municipality_and_level'})
train_data = train_data.merge(municipalities, how="left", on=["municipality_name", "lv2"])
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')


In [ ]:
# train_data['num_stores_within_distance'] = 0
# for index in range(len(train_data)):
#     lvl1 = train_data._get_value(index,'lv2')
#     lat1 = train_data._get_value(index,'lat')
#     lon1 = train_data._get_value(index,'lon')
#     count = 0
#     for inx in range(len(train_data)):
#         lvl2 = train_data._get_value(inx,'lv2')
#         lat2 = train_data._get_value(inx,'lat')
#         lon2 = train_data._get_value(inx,'lon')
#         try:
#             dist = geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km #some values have nan in them. must be fixed. 
            
#         except:
#             dist = 0

#         if dist < 2 and lvl1 == lvl2:
#             count += 1
#     train_data._set_value(index, 'num_stores_within_distance', count- 1)

In [ ]:
# train_data['mean_rev_of_stores_within_distance_same_lv2'] = 0
# df = pd.DataFrame(columns=['store_id','revenue'])
# for index in range(len(train_data)):
#     lvl1 = train_data._get_value(index,'lv2')
#     lat1 = train_data._get_value(index,'lat')
#     lon1 = train_data._get_value(index,'lon')
#     store1 = train_data._get_value(index,'store_id')
#     for inx in range(len(train_data)):
#         lvl2 = train_data._get_value(inx,'lv2')
#         lat2 = train_data._get_value(inx,'lat')
#         lon2 = train_data._get_value(inx,'lon')
#         store2 = train_data._get_value(inx,'store_id')
#         try:
#             dist = geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km #some values have nan in them. must be fixed. 
            
#         except:
#             dist = 0

#         if dist < 2 and lvl1 == lvl2 and store1 != store2:
#             df.loc[len(df.index)] = [store2, train_data._get_value(inx,'revenue')]
#     mean_rev = df["revenue"].mean()
#     train_data._set_value(index, 'mean_rev_of_stores_within_distance_same_lv2', mean_rev)



In [ ]:
# train_data['distance_to_closest_buss_station'] = 0
# train_data['importance_level'] = 0
# train_data['closest_local_knutepunkt'] = 0

# gdf = gpd.GeoDataFrame(busstops)
# gdf['geometry'] = gpd.GeoSeries.from_wkt(gdf['geometry'])

# for index in range(len(gdf)):
#     closest = 100000 #large number
#     importance = ""
#     closest_high_importance = 0
#     lat = train_data._get_value(index,'lat')
#     lon = train_data._get_value(index,'lon')
#     for numbuss in range(len(gdf)):
#         val = gdf._get_value(index,'geometry')
#         dist = geopy.distance.geodesic((val.x, val.y), (lat, lon)).km
#         if dist < closest:
#             closest = dist
#             importance = gdf._get_value(index,'importance_level')
#         if gdf._get_value(index,'importance_level') == 'Lokalt knutepunkt':
#             closest_high_importance = dist
#     train_data._set_value(index, 'distance_to_closest_buss_station', closest)
#     train_data._set_value(index, 'importance_level', importance)
#     train_data._set_value(index, 'closest_local_knutepunkt', closest_high_importance)


In [ ]:
col_at_end = ['revenue']
train_data = train_data[[c for c in train_data if c not in col_at_end] + [c for c in col_at_end if c in train_data]]

In [ ]:
train_data.head()

In [ ]:
print(list(train_data.columns))